![dvd_image](dvd_image.jpg)

A DVD rental company needs your help! They want to figure out how many days a customer will rent a DVD for based on some features and has approached you for help. They want you to try out some regression models which will help predict the number of days a customer will rent a DVD for. The company wants a model which yeilds a MSE of 3 or less on a test set. The model you make will help the company become more efficient inventory planning.

The data they provided is in the csv file `rental_info.csv`. It has the following features:
- `"rental_date"`: The date (and time) the customer rents the DVD.
- `"return_date"`: The date (and time) the customer returns the DVD.
- `"amount"`: The amount paid by the customer for renting the DVD.
- `"amount_2"`: The square of `"amount"`.
- `"rental_rate"`: The rate at which the DVD is rented for.
- `"rental_rate_2"`: The square of `"rental_rate"`.
- `"release_year"`: The year the movie being rented was released.
- `"length"`: Lenght of the movie being rented, in minuites.
- `"length_2"`: The square of `"length"`.
- `"replacement_cost"`: The amount it will cost the company to replace the DVD.
- `"special_features"`: Any special features, for example trailers/deleted scenes that the DVD also has.
- `"NC-17"`, `"PG"`, `"PG-13"`, `"R"`: These columns are dummy variables of the rating of the movie. It takes the value 1 if the move is rated as the column name and 0 otherwise. For your convinience, the reference dummy has already been dropped.

In [12]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE

from sklearn.linear_model import Lasso

from sklearn.linear_model import LinearRegression

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV

# Read the csv file
rental_info = pd.read_csv('rental_info.csv')

# we will encounter an error, if we don't use pd.to_datetime since performing calculations on a string is not allowed. The output should in days
rental_info['rental_length_days'] = (pd.to_datetime(rental_info['return_date']) - pd.to_datetime(rental_info['rental_date'])).dt.days

# To add dummy variables based on the entries we'll use np.where() which works as an if-else statement but for arrays np.where(condition, value_if_true, value_if_false)
rental_info['deleted_scenes'] = np.where(rental_info['special_features'].str.contains("Deleted Scenes"), 1, 0)
rental_info['behind_the_scenes'] = np.where(rental_info['special_features'].str.contains("Behind the Scenes"), 1, 0)

# Drop columns that may leak data about the target or cause noise
X = rental_info.drop(['rental_date', 'return_date', 'rental_length_days', 'special_features'], axis=1)
y = rental_info['rental_length_days']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=9)

# Lasso is used for variable selection and regularization 
lasso = Lasso(alpha=0.4, random_state=9)
lasso.fit(X_train, y_train)
lasso_coef = lasso.coef_

# Identify best performing features using the iloc method
X_lasso_train, X_lasso_test = X_train.iloc[:, lasso_coef > 0], X_test.iloc[:, lasso_coef > 0]

# A regression model trained on the lasso-selected data 
lr = LinearRegression()
lr.fit(X_lasso_train, y_train)
y_pred = lr.predict(X_lasso_test)
mse_for_LassoReg = MSE(y_test, y_pred)
print(mse_for_LassoReg)

# Hyperparameters tuning for random forest
param_rf = {'n_estimators': np.arange(1, 101, 1),
              'max_depth': np.arange(1, 11, 1)}
rf = RandomForestRegressor()
rand_search = RandomizedSearchCV(rf, 
                                 param_distributions=param_rf, 
                                 cv=5, random_state=9)
rand_search.fit(X_train, y_train)
hyper_params = rand_search.best_params_
print(hyper_params)

# Create a random forest with the best hyperparameters
rf = RandomForestRegressor(n_estimators=hyper_params["n_estimators"], 
                           max_depth=hyper_params["max_depth"], 
                           random_state=9)

# Random Forest can perform feature importance evaluation on its own without the need of prior lasso-feature selection to perform well. That's why this model isn't trained on the lasso-features
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)
mse_random_forest = MSE(y_test, rf_pred)
print(rf)
print(mse_random_forest)

# Selecting the Best Model which is the last one (random forest) since it has a lower mse
best_model = rf
best_mse = mse_random_forest

4.812297241276244
{'n_estimators': 51, 'max_depth': 10}
RandomForestRegressor(max_depth=10, n_estimators=51, random_state=9)
2.225667528098759
